## Imports

In [2]:
# Data handling
import pandas as pd
import numpy as np
import networkx as nx

# Machine Learning
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
import giotto as gio
import giotto.time_series as ts
import giotto.graphs as gr
import giotto.diagrams as diag
import giotto.homology as hl

# Plotting functions
import matplotlib.pyplot as plt
import seaborn as sns
from plotting import plot_diagram, plot_landscapes
from plotting import plot_betti_surfaces, plot_betti_curves
from plotting import plot_point_cloud
from xgboost import plot_importance

# Others
import os, sys
sys.path.append('..')
from itertools import product
import time
from dataset_creation import *
import pickle

In [117]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sympy.geometry import Point3D



def plot_molecule(molecule_name, structures_df):
    #from here: https://www.kaggle.com/mykolazotko/3d-visualization-of-molecules-with-plotly
    """Creates a 3D plot of the molecule"""
    
    atomic_radii = dict(C=0.77, F=0.71, H=0.38, N=0.75, O=0.73)  
    cpk_colors = dict(C='black', F='green', H='white', N='blue', O='red')

    molecule = structures_df[structures_df.molecule_name == molecule_name]
    coordinates = molecule[['x', 'y', 'z']].values
    x_coordinates = coordinates[:, 0]
    y_coordinates = coordinates[:, 1]
    z_coordinates = coordinates[:, 2]
    elements = molecule.atom.tolist()
    radii = [atomic_radii[element] for element in elements]
    
    def get_bonds():
        """Generates a set of bonds from atomic cartesian coordinates"""
        ids = np.arange(coordinates.shape[0])
        bonds = dict()
        coordinates_compare, radii_compare, ids_compare = coordinates, radii, ids
        
        for _ in range(len(ids)):
            coordinates_compare = np.roll(coordinates_compare, -1, axis=0)
            radii_compare = np.roll(radii_compare, -1, axis=0)
            ids_compare = np.roll(ids_compare, -1, axis=0)
            distances = np.linalg.norm(coordinates - coordinates_compare, axis=1)
            bond_distances = (radii + radii_compare) * 1.3
            mask = np.logical_and(distances > 0.1, distances <  bond_distances)
            distances = distances.round(2)
            new_bonds = {frozenset([i, j]): dist for i, j, dist in zip(ids[mask], ids_compare[mask], distances[mask])}
            bonds.update(new_bonds)
        return bonds            
            
    def atom_trace():
        """Creates an atom trace for the plot"""
        colors = [cpk_colors[element] for element in elements]
        markers = dict(color=colors, line=dict(color='lightgray', width=2), size=7, symbol='circle', opacity=0.8)
        trace = go.Scatter3d(x=x_coordinates, y=y_coordinates, z=z_coordinates, mode='markers', marker=markers,
                             text=elements, name='')
        return trace

    def bond_trace():
        """"Creates a bond trace for the plot"""
        trace = go.Scatter3d(x=[], y=[], z=[], hoverinfo='none', mode='lines',
                             marker=dict(color='grey', size=7, opacity=1))
        for i, j in bonds.keys():
            trace['x'] += (x_coordinates[i], x_coordinates[j], None)
            trace['y'] += (y_coordinates[i], y_coordinates[j], None)
            trace['z'] += (z_coordinates[i], z_coordinates[j], None)
        return trace
    
    bonds = get_bonds()
    
    zipped = zip(range(len(elements)), x_coordinates, y_coordinates, z_coordinates)
    annotations_id = [dict(text=num, x=x, y=y, z=z, showarrow=False, yshift=15, font = dict(color = "blue"))
                   for num, x, y, z in zipped]
    
    annotations_length = []
    for (i, j), dist in bonds.items():
        p_i, p_j = Point3D(coordinates[i]), Point3D(coordinates[j])
        p = p_i.midpoint(p_j)
        annotation = dict(text=dist, x=float(p.x), y=float(p.y), z=float(p.z), showarrow=False, yshift=15)
        annotations_length.append(annotation)   
    
    updatemenus = list([
        dict(buttons=list([
                 dict(label = 'Atom indices',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_id}]),
                 dict(label = 'Bond lengths',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_length}]),
                 dict(label = 'Atom indices & Bond lengths',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_id + annotations_length}]),
                 dict(label = 'Hide all',
                      method = 'relayout',
                      args = [{'scene.annotations': []}])
                 ]),
                 direction='down',
                 xanchor = 'left',
                 yanchor = 'top'
            ),        
    ])
    
    data = [atom_trace(), bond_trace()]
    axis_params = dict(showgrid=False, showbackground=False, showticklabels=False, zeroline=False, titlefont=dict(color='white'))
    layout = dict(scene=dict(xaxis=axis_params, yaxis=axis_params, zaxis=axis_params, annotations=annotations_id), 
                  margin=dict(r=0, l=0, b=0, t=0), showlegend=True, updatemenus=updatemenus)

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [201]:
import plotly.graph_objs as go
from sympy.geometry import Point3D

def my_plot_molecule(molecule_name, structures_df):
    radius = dict(C=0.77, F=0.71, H=0.38, N=0.75, O=0.73)
    element_colors = dict(C='black', F='green', H='white', N='blue', O='red')
    
    molecule_df = structures_df[structures_df['molecule_name'] == molecule_name]
    x = molecule_df['x'].values
    y = molecule_df['y'].values
    z = molecule_df['z'].values
    elements = molecule_df['atom'].values
    r = [radius[e] for e in elements]
    coordinates = pd.DataFrame([x,y,z]).T
    
    def get_bonds():
        """Generates a set of bonds from atomic cartesian coordinates"""
        ids = np.arange(coordinates.shape[0])
        bonds = dict()
        coordinates_compare, radii_compare, ids_compare = coordinates, r, ids
        
        for _ in range(len(ids)):
            coordinates_compare = np.roll(coordinates_compare, -1, axis=0)
            radii_compare = np.roll(radii_compare, -1, axis=0)
            ids_compare = np.roll(ids_compare, -1, axis=0)
            distances = np.linalg.norm(coordinates - coordinates_compare, axis=1)
            bond_distances = (r + radii_compare) * 1.3
            mask = np.logical_and(distances > 0.1, distances <  bond_distances)
            distances = distances.round(2)
            new_bonds = {frozenset([i, j]): dist for i, j, dist in zip(ids[mask], ids_compare[mask], distances[mask])}
            bonds.update(new_bonds)
        return bonds       
    
    def get_bond_trace():
        bond_trace = go.Scatter3d(x=[], y=[], z=[], hoverinfo='none', mode='lines',
                             marker=dict(color='grey', size=7, opacity=1))
        for i,j in bonds.keys():
            bond_trace['x'] += (x[i], x[j], None)
            bond_trace['y'] += (y[i], y[j], None)
            bond_trace['z'] += (z[i], z[j], None)
        return bond_trace
    
    def get_atom_trace():
        """Creates an atom trace for the plot"""
        colors = [element_colors[element] for element in elements]
        markers = dict(color=colors, line=dict(color='lightgray', width=2), size=10, symbol='circle', opacity=0.8)
        trace = go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=markers,
                             text=elements, name='', hoverlabel=dict(bgcolor=colors))
        return trace
    
    bonds = get_bonds()
    annotations_length = []
    for (i, j), dist in bonds.items():
        p_i, p_j = Point3D(coordinates.values[i]), Point3D(coordinates.values[j])
        p = p_i.midpoint(p_j)
        annotation = dict(text=dist, x=float(p.x), y=float(p.y), z=float(p.z), showarrow=False, yshift=15)
        annotations_length.append(annotation) 
    data = [get_atom_trace(), get_bond_trace()]
    
    axis_params = dict(showgrid=False, showbackground=False, showticklabels=False, zeroline=False, 
                       titlefont=dict(color='white'))
    layout = dict(scene=dict(xaxis=axis_params, yaxis=axis_params, zaxis=axis_params), 
                  margin=dict(r=0, l=0, b=0, t=0), showlegend=False, annotations=[
                        go.layout.Annotation(
                            text='Molecule Name:<br>{}'.format(molecule_name),
                            align='left',
                            showarrow=False,
                            xref='paper',
                            yref='paper',
                            x=0.95,
                            y=0.95,
                            bordercolor='black',
                            borderwidth=1
                        )
                    ])
    
    fig = go.Figure(data=data, layout=layout)
    
    return fig

In [200]:
#plot_molecule('dsgdb9nsd_000008', structures)
my_plot_molecule('dsgdb9nsd_000303', structures)

## Import Data

In [7]:
X = pd.read_pickle('../data/generated_features/X_1.pickle')
y = pd.read_pickle('../data/generated_features/y_1.pickle')

X.columns

Index(['type', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1',
       'dist', 'dist_x', 'dist_y', 'dist_z', 'type_0', 'type_1',
       'dist_to_type_mean', 'dist_to_type_0_mean', 'dist_to_type_1_mean',
       'molecule_type_dist_mean', 'num_tda_1', 'time_tda_0', 'time_tda_1',
       'amp_tda', 'graph_holes_0', 'graph_holes_1', 'graph_lifetime_1',
       'graph_amplitude_1'],
      dtype='object')

In [8]:
file_folder = '../data/champs-scalar-coupling' if 'champs-scalar-coupling' in os.listdir('../data/') else '../data'
os.listdir(file_folder)

train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
sub = pd.read_csv(f'{file_folder}/sample_submission.csv')
structures = pd.read_csv(f'{file_folder}/structures.csv')
contributions = pd.read_csv((f'{file_folder}/scalar_coupling_contributions.csv'))

## Create new features

In [198]:
list(structures['molecule_name'].unique())

['dsgdb9nsd_000001',
 'dsgdb9nsd_000002',
 'dsgdb9nsd_000003',
 'dsgdb9nsd_000004',
 'dsgdb9nsd_000005',
 'dsgdb9nsd_000007',
 'dsgdb9nsd_000008',
 'dsgdb9nsd_000009',
 'dsgdb9nsd_000010',
 'dsgdb9nsd_000011',
 'dsgdb9nsd_000012',
 'dsgdb9nsd_000013',
 'dsgdb9nsd_000014',
 'dsgdb9nsd_000015',
 'dsgdb9nsd_000016',
 'dsgdb9nsd_000017',
 'dsgdb9nsd_000018',
 'dsgdb9nsd_000019',
 'dsgdb9nsd_000020',
 'dsgdb9nsd_000021',
 'dsgdb9nsd_000022',
 'dsgdb9nsd_000023',
 'dsgdb9nsd_000024',
 'dsgdb9nsd_000026',
 'dsgdb9nsd_000027',
 'dsgdb9nsd_000028',
 'dsgdb9nsd_000029',
 'dsgdb9nsd_000030',
 'dsgdb9nsd_000031',
 'dsgdb9nsd_000032',
 'dsgdb9nsd_000033',
 'dsgdb9nsd_000034',
 'dsgdb9nsd_000035',
 'dsgdb9nsd_000036',
 'dsgdb9nsd_000037',
 'dsgdb9nsd_000038',
 'dsgdb9nsd_000039',
 'dsgdb9nsd_000040',
 'dsgdb9nsd_000041',
 'dsgdb9nsd_000042',
 'dsgdb9nsd_000043',
 'dsgdb9nsd_000044',
 'dsgdb9nsd_000045',
 'dsgdb9nsd_000046',
 'dsgdb9nsd_000047',
 'dsgdb9nsd_000048',
 'dsgdb9nsd_000049',
 'dsgdb9nsd_0